> ### EEE4423: Deep Learning Lab

# Final Project: Long-tail Visual Recognition for Image Classification

<h4><div style="text-align: right"> Due date: June 24, 2022.  </div> <br>
<div style="text-align: right"> Please upload your file @ LearnUs and submit via e-mail by 2 PM in the form of [ID_Name_project.ipynb]. </div></h4>

이 파일은 정확도의 경향성을 보기 위한 파일이다.

In [27]:
import os
import os.path as osp
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from misc.project.utils import resnet18, IMBALANCECIFAR10, IMBALANCECIFAR100, compute_accuracy
from torch.nn.parameter import Parameter

import warnings
warnings.filterwarnings("ignore", category=UserWarning) ## 경고 무시

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [28]:
print('STEP 6: INSTANTIATE OPTIMIZER CLASS')

DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step'] # 질문
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/featuresampling/CIFAR10/exp-0.1' 
LR = 0.1
BATCH_SIZE = 128 
MOMENTUM = 0.9
WEIGHT_DECAY = 2e-4
EPOCHS = 100

STEP 6: INSTANTIATE OPTIMIZER CLASS


In [29]:
os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [30]:
print('STEP 1: LOADING DATASET')

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

if DATASET == 'CIFAR10':
    train_dataset_0 = IMBALANCECIFAR10(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, train=True, download=True, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR10(root='../dataset/project', train=False, download=True, transform=transform_test)
elif DATASET == 'CIFAR100':
    train_dataset_0 = IMBALANCECIFAR100(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, train=True, download=True, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR100(root='../dataset/project', train=False, download=True, transform=transform_test)

cls_num_list = train_dataset_0.get_cls_num_list() # 각 class에 몇개가 들어있는지 나타냄 
print('cls num list:')
print(cls_num_list)
num_classes = len(cls_num_list)
print(num_classes)

STEP 1: LOADING DATASET
Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 3871, 2997, 2320, 1796, 1391, 1077, 834, 645, 500]
10


In [31]:
from torch.utils.data import Dataset, DataLoader, random_split
torch.manual_seed(torch.initial_seed())
generator = torch.Generator()
generator.manual_seed(0)
dataset_size = len(train_dataset_0)
validation_size = int(dataset_size * 0.1) if not int(dataset_size * 0.1)==0 else 1 
train_size = dataset_size - validation_size
test_size = 0

train_dataset, validation_dataset, test_dataset_0 = random_split(train_dataset_0, [train_size, validation_size, test_size],generator=generator)

print(len(train_dataset_0))
print(len(train_dataset))
print(len(validation_dataset))
print(len(test_dataset_0))

20431
18388
2043
0


여기서 진행하는 것은 re-sampling이다.

In [32]:
# data sampler 구현 

import random
import numpy as np
from torch.utils.data.sampler import Sampler
import pdb

# data 중에 random하게 하나를 뽑음.
class RandomCycleIter:
    def __init__ (self, data, test_mode=False):
        self.data_list = list(data)
        self.length = len(self.data_list)
        self.i = self.length - 1
        self.test_mode = test_mode
    def __iter__ (self):
        return self
    def __next__ (self):
        self.i += 1
        if self.i == self.length:
            self.i = 0
            if not self.test_mode:
                random.shuffle(self.data_list)
        return self.data_list[self.i]
    
def class_aware_sample_generator (cls_iter, data_iter_list, n, num_samples_cls=1):
    i = 0
    j = 0
    while i < n:
        if j >= num_samples_cls:
            j = 0
        if j == 0:
            temp_tuple = next(zip(*[data_iter_list[next(cls_iter)]]*num_samples_cls))
            yield temp_tuple[j]
        else:
            yield temp_tuple[j]
        i += 1
        j += 1

class ClassAwareSampler (Sampler):
    def __init__(self, data_source, num_samples_cls=1,):
        labels = []
        for i in range(len(train_dataset)) :
            labels.append(train_dataset[i][1])
        num_classes = 10
        self.class_iter = RandomCycleIter(range(num_classes))
        cls_data_list = [list() for _ in range(num_classes)] # [[], [], [], [], [], [], [], [], [], []]
        for i, label in enumerate(labels):
            cls_data_list[label].append(i)
        self.data_iter_list = [RandomCycleIter(x) for x in cls_data_list]
        # print(self.data_iter_list)
        self.num_samples = max([len(x) for x in cls_data_list]) * len(cls_data_list)
        self.num_samples_cls = num_samples_cls
    def __iter__ (self):
        return class_aware_sample_generator(self.class_iter, self.data_iter_list,
                                            self.num_samples, self.num_samples_cls)
    def __len__ (self):
        return self.num_samples

In [33]:
print('STEP 2: MAKING DATASET ITERABLE')
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=None,
    num_workers=4, drop_last=False, sampler = ClassAwareSampler(train_dataset,4))

val_loader = torch.utils.data.DataLoader(
    validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=4, drop_last=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=100, shuffle=False,
    num_workers=4, )

STEP 2: MAKING DATASET ITERABLE


In [9]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=None,
    num_workers=4, drop_last=False, sampler = ClassAwareSampler(train_dataset,4))

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=100, shuffle=False,
    num_workers=4, )

In [10]:
# 다른 방법이긴하지만 이렇게 표현가능  
list1 = np.zeros(10)
for batch_index, data in enumerate(train_loader):
    image, target = data
    for j in range(len(target)) :
        k =target[j] 
        list1[k]+=1

print(list1)
print(list1.sum())

[5000. 5000. 5000. 5000. 5000. 5000. 5000. 5000. 5000. 5000.]
50000.0


In [34]:
print('STEP 3: CREATE MODEL CLASS (VGG16)')

class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18, self).__init__()
        self.backbone = resnet18()
        self.classifier = nn.Linear(512, num_classes)
    
    def forward(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = F.adaptive_max_pool2d(x, 1)
        x = x.view(batch_size, -1)
        pred = self.classifier(x)
#         print(pred.size()) # torch.Size([128, 10])
        return pred

STEP 3: CREATE MODEL CLASS (VGG16)


In [35]:
print('STEP 4: INSTANTIATE MODEL CLASS')

model = ResNet18(num_classes)
model = model.cuda()

print('STEP 5: INSTANTIATE LOSS CLASS')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

STEP 4: INSTANTIATE MODEL CLASS
STEP 5: INSTANTIATE LOSS CLASS


In [36]:
torch.cuda.empty_cache()

In [37]:
for epoch in range(EPOCHS):
    loss_history = []
    model.train()
    for batch_index, data in enumerate(train_loader):
        image, target = data
        image, target = image.cuda(), target.cuda()

        pred = model(image)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())

    topk_acc, head_acc, tail_acc = compute_accuracy(val_loader, model)
    loss_mean = np.mean(loss_history)
    scheduler.step()

    print('Epoch: [{:03d}] \t Loss {:.4f} \t Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(epoch+1, loss_mean, topk_acc[0], head_acc[0], tail_acc[0]))

Epoch: [001] 	 Loss 2.0046 	 Acc 39.35 	 AccHead 38.36 	 AccTail 43.16
Epoch: [002] 	 Loss 1.4182 	 Acc 51.49 	 AccHead 53.18 	 AccTail 45.05
Epoch: [003] 	 Loss 1.2171 	 Acc 56.58 	 AccHead 55.71 	 AccTail 59.91
Epoch: [004] 	 Loss 1.0492 	 Acc 58.49 	 AccHead 57.01 	 AccTail 64.15
Epoch: [005] 	 Loss 0.9340 	 Acc 62.56 	 AccHead 62.45 	 AccTail 62.97
Epoch: [006] 	 Loss 0.8414 	 Acc 61.72 	 AccHead 62.38 	 AccTail 59.20
Epoch: [007] 	 Loss 0.7743 	 Acc 68.82 	 AccHead 70.66 	 AccTail 61.79
Epoch: [008] 	 Loss 0.7146 	 Acc 70.93 	 AccHead 74.12 	 AccTail 58.73
Epoch: [009] 	 Loss 0.6637 	 Acc 70.44 	 AccHead 70.41 	 AccTail 70.52
Epoch: [010] 	 Loss 0.6219 	 Acc 64.46 	 AccHead 62.75 	 AccTail 70.99
Epoch: [011] 	 Loss 0.5839 	 Acc 71.27 	 AccHead 72.58 	 AccTail 66.27
Epoch: [012] 	 Loss 0.5545 	 Acc 73.08 	 AccHead 73.69 	 AccTail 70.75
Epoch: [013] 	 Loss 0.5302 	 Acc 72.83 	 AccHead 73.38 	 AccTail 70.75
Epoch: [014] 	 Loss 0.5025 	 Acc 70.93 	 AccHead 71.65 	 AccTail 68.16
Epoch:

In [38]:
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': epoch},
    osp.join(SAVE_DIR, 'ep{:03d}.pth'.format(epoch+1))
)

In [39]:
##  CIFAR 10 : exp : 0.1 
SAVE_DIR = 'logs/Tendency/featuresampling/CIFAR10/exp-0.1' 
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 72.91 	 AccHead 74.94 	 AccTail 70.88


baseline

In [40]:
SAVE_DIR = 'logs/Tendency/baseline/CIFAR10/exp-0.1' 

In [41]:
print('STEP 2: MAKING DATASET ITERABLE')
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=None,
    num_workers=4, drop_last=False)

val_loader = torch.utils.data.DataLoader(
    validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=4, drop_last=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=100, shuffle=False,
    num_workers=4, )

STEP 2: MAKING DATASET ITERABLE


In [42]:
print('STEP 4: INSTANTIATE MODEL CLASS')

model = ResNet18(num_classes)
model = model.cuda()

print('STEP 5: INSTANTIATE LOSS CLASS')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

STEP 4: INSTANTIATE MODEL CLASS
STEP 5: INSTANTIATE LOSS CLASS


In [43]:
torch.cuda.empty_cache()

In [44]:
for epoch in range(EPOCHS):
    loss_history = []
    model.train()
    for batch_index, data in enumerate(train_loader):
        image, target = data
        image, target = image.cuda(), target.cuda()

        pred = model(image)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())

    topk_acc, head_acc, tail_acc = compute_accuracy(val_loader, model)
    loss_mean = np.mean(loss_history)
    scheduler.step()

    print('Epoch: [{:03d}] \t Loss {:.4f} \t Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(epoch+1, loss_mean, topk_acc[0], head_acc[0], tail_acc[0]))

Epoch: [001] 	 Loss 2.5109 	 Acc 41.65 	 AccHead 52.56 	 AccTail 0.00
Epoch: [002] 	 Loss 1.6600 	 Acc 45.03 	 AccHead 54.35 	 AccTail 9.43
Epoch: [003] 	 Loss 1.5383 	 Acc 47.87 	 AccHead 54.35 	 AccTail 23.11
Epoch: [004] 	 Loss 1.4318 	 Acc 50.27 	 AccHead 57.38 	 AccTail 23.11
Epoch: [005] 	 Loss 1.3672 	 Acc 53.35 	 AccHead 60.90 	 AccTail 24.53
Epoch: [006] 	 Loss 1.3262 	 Acc 54.14 	 AccHead 61.58 	 AccTail 25.71
Epoch: [007] 	 Loss 1.2877 	 Acc 56.68 	 AccHead 63.68 	 AccTail 29.95
Epoch: [008] 	 Loss 1.2077 	 Acc 55.21 	 AccHead 61.40 	 AccTail 31.60
Epoch: [009] 	 Loss 1.1439 	 Acc 59.91 	 AccHead 67.08 	 AccTail 32.55
Epoch: [010] 	 Loss 1.0994 	 Acc 62.41 	 AccHead 68.99 	 AccTail 37.26
Epoch: [011] 	 Loss 1.0377 	 Acc 62.26 	 AccHead 67.70 	 AccTail 41.51
Epoch: [012] 	 Loss 0.9979 	 Acc 64.12 	 AccHead 68.93 	 AccTail 45.75
Epoch: [013] 	 Loss 0.9654 	 Acc 65.10 	 AccHead 71.90 	 AccTail 39.15
Epoch: [014] 	 Loss 0.9290 	 Acc 64.86 	 AccHead 69.92 	 AccTail 45.52
Epoch: [

In [46]:
os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [48]:
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': epoch},
    osp.join(SAVE_DIR, 'ep{:03d}.pth'.format(epoch+1))
)

In [49]:
##  CIFAR 10 : exp : 0.1 
SAVE_DIR = 'logs/Tendency/baseline/CIFAR10/exp-0.1' 
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 67.52 	 AccHead 73.42 	 AccTail 61.62


여기서부터는 LWS 기법이다.

In [50]:
SAVE_DIR = 'logs/Tendency/LWS/CIFAR10/exp-0.1' 

In [51]:
print('STEP 3: CREATE MODEL CLASS (VGG16)')

class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18, self).__init__()
        self.backbone = resnet18()
        self.classifier = nn.Linear(512, num_classes)
        self.scales = Parameter(torch.ones(num_classes))
    
    def forward(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = F.adaptive_max_pool2d(x, 1)
        x = x.view(batch_size, -1)
        pred = self.classifier(x)
        # LWS 구현
        pred *= self.scales
#         print(pred.size()) # torch.Size([128, 10])
        return pred

STEP 3: CREATE MODEL CLASS (VGG16)


In [52]:
print('STEP 4: INSTANTIATE MODEL CLASS')

model = ResNet18(num_classes)
model = model.cuda()

print('STEP 5: INSTANTIATE LOSS CLASS')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

STEP 4: INSTANTIATE MODEL CLASS
STEP 5: INSTANTIATE LOSS CLASS


In [53]:
torch.cuda.empty_cache()

In [54]:
for epoch in range(EPOCHS):
    loss_history = []
    model.train()
    for batch_index, data in enumerate(train_loader):
        image, target = data
        image, target = image.cuda(), target.cuda()

        pred = model(image)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())

    topk_acc, head_acc, tail_acc = compute_accuracy(val_loader, model)
    loss_mean = np.mean(loss_history)
    scheduler.step()

    print('Epoch: [{:03d}] \t Loss {:.4f} \t Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(epoch+1, loss_mean, topk_acc[0], head_acc[0], tail_acc[0]))

Epoch: [001] 	 Loss 3.0158 	 Acc 32.11 	 AccHead 40.52 	 AccTail 0.00
Epoch: [002] 	 Loss 1.7633 	 Acc 41.80 	 AccHead 52.75 	 AccTail 0.00
Epoch: [003] 	 Loss 1.6069 	 Acc 44.20 	 AccHead 55.34 	 AccTail 1.65
Epoch: [004] 	 Loss 1.5288 	 Acc 46.06 	 AccHead 55.53 	 AccTail 9.91
Epoch: [005] 	 Loss 1.4778 	 Acc 48.31 	 AccHead 56.08 	 AccTail 18.63
Epoch: [006] 	 Loss 1.4209 	 Acc 50.91 	 AccHead 58.55 	 AccTail 21.70
Epoch: [007] 	 Loss 1.3588 	 Acc 52.86 	 AccHead 60.84 	 AccTail 22.41
Epoch: [008] 	 Loss 1.3142 	 Acc 52.86 	 AccHead 59.48 	 AccTail 27.59
Epoch: [009] 	 Loss 1.2715 	 Acc 54.38 	 AccHead 61.15 	 AccTail 28.54
Epoch: [010] 	 Loss 1.2255 	 Acc 56.93 	 AccHead 63.50 	 AccTail 31.84
Epoch: [011] 	 Loss 1.1729 	 Acc 57.17 	 AccHead 63.93 	 AccTail 31.37
Epoch: [012] 	 Loss 1.1222 	 Acc 60.16 	 AccHead 67.33 	 AccTail 32.78
Epoch: [013] 	 Loss 1.0824 	 Acc 62.80 	 AccHead 68.44 	 AccTail 41.27
Epoch: [014] 	 Loss 1.0331 	 Acc 61.14 	 AccHead 67.33 	 AccTail 37.50
Epoch: [01

In [55]:
os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [56]:
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': epoch},
    osp.join(SAVE_DIR, 'ep{:03d}.pth'.format(epoch+1))
)

In [57]:
##  CIFAR 10 : exp : 0.1 
SAVE_DIR = 'logs/Tendency/LWS/CIFAR10/exp-0.1' 
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 67.40 	 AccHead 72.84 	 AccTail 61.96


여기서 부터는 mymodel이다. 

In [58]:
SAVE_DIR = 'logs/Tendency/mymodel/CIFAR10/exp-0.1' 

In [60]:
print('STEP 4: INSTANTIATE MODEL CLASS')

model_head = ResNet18(num_classes)
model_tail = ResNet18(num_classes)
model_head = model_head.cuda()
model_tail = model_tail.cuda()

print('STEP 5: INSTANTIATE LOSS CLASS')
criterion = nn.CrossEntropyLoss()
optimizer_head = torch.optim.SGD(model_head.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
optimizer_tail = torch.optim.SGD(model_tail.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler_head = torch.optim.lr_scheduler.StepLR(optimizer_head, step_size=150, gamma=0.1)
scheduler_tail = torch.optim.lr_scheduler.StepLR(optimizer_tail, step_size=150, gamma=0.1)

STEP 4: INSTANTIATE MODEL CLASS
STEP 5: INSTANTIATE LOSS CLASS


In [61]:
torch.cuda.empty_cache()

In [62]:
print('STEP 7: TRAIN THE MODEL')
cri=num_classes//2-1
max_norm = 5

for epoch in range(EPOCHS):
    loss_history = []
    # train
    model_head.train()
    model_tail.train()
    for batch_index, data in enumerate(train_loader):
        image, target = data
        image, target = image.cuda(), target.cuda()
        
        image_tail = image[target>cri]
        target_tail = target[target>cri]
        image_head = image[target<=cri]
        target_head = target[target<=cri]
            
        pred_head = model_head(image_head)
        loss_head = criterion(pred_head, target_head)
        optimizer_head.zero_grad()
        loss_head.backward()
        optimizer_head.step()
        
        if target_tail.size()[0] == 1 :
            image_tail=torch.cat([image_tail, image_tail])
            target_tail = torch.cat([target_tail, target_tail])
            
        if target_tail.size()[0] == 0 :
            loss = loss_head
#             print('ok')
        else :
            pred_tail = model_tail(image_tail)
            loss_tail = criterion(pred_tail, target_tail)
            optimizer_tail.zero_grad()
            optimizer_tail.zero_grad()
            loss_tail.backward()
            torch.nn.utils.clip_grad_norm_(model_tail.parameters(), max_norm)
            optimizer_tail.step()     
            loss = loss_head+loss_tail

        loss_history.append(loss.item())
        
    # eval
    topk_acc, head_acc_ok, tail_acc = compute_accuracy(val_loader, model_head)
    topk_acc, head_acc, tail_acc_ok = compute_accuracy(val_loader, model_tail)
    
    topk_acc = (head_acc_ok[0]+ tail_acc_ok[0])/2
    tail_acc = tail_acc_ok[0]
    head_acc = head_acc_ok[0]
    
    loss_mean = np.mean(loss_history)
    scheduler_head.step()
    scheduler_tail.step()

    print('Epoch: [{:03d}] \t Loss {:.4f} \t Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(epoch+1, loss_mean, topk_acc, head_acc, tail_acc))

STEP 7: TRAIN THE MODEL
Epoch: [001] 	 Loss 4.3781 	 Acc 39.31 	 AccHead 44.66 	 AccTail 33.96
Epoch: [002] 	 Loss 2.6075 	 Acc 50.38 	 AccHead 56.89 	 AccTail 43.87
Epoch: [003] 	 Loss 2.3109 	 Acc 54.40 	 AccHead 61.40 	 AccTail 47.41
Epoch: [004] 	 Loss 2.1666 	 Acc 58.87 	 AccHead 64.67 	 AccTail 53.07
Epoch: [005] 	 Loss 2.0282 	 Acc 62.12 	 AccHead 66.46 	 AccTail 57.78
Epoch: [006] 	 Loss 1.8902 	 Acc 65.48 	 AccHead 67.51 	 AccTail 63.44
Epoch: [007] 	 Loss 1.8220 	 Acc 65.88 	 AccHead 67.14 	 AccTail 64.62
Epoch: [008] 	 Loss 1.7210 	 Acc 66.83 	 AccHead 69.98 	 AccTail 63.68
Epoch: [009] 	 Loss 1.6418 	 Acc 68.98 	 AccHead 71.22 	 AccTail 66.75
Epoch: [010] 	 Loss 1.5464 	 Acc 71.63 	 AccHead 73.44 	 AccTail 69.81
Epoch: [011] 	 Loss 1.5074 	 Acc 73.55 	 AccHead 74.92 	 AccTail 72.17
Epoch: [012] 	 Loss 1.4389 	 Acc 70.68 	 AccHead 74.61 	 AccTail 66.75
Epoch: [013] 	 Loss 1.3983 	 Acc 73.66 	 AccHead 77.52 	 AccTail 69.81
Epoch: [014] 	 Loss 1.3497 	 Acc 74.07 	 AccHead 75.7

In [64]:
os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [66]:
torch.save({
    'model_head': model_head.state_dict(),
    'optimizer_head': optimizer_head.state_dict(),
    'model_tail': model_tail.state_dict(),
    'optimizer_tail': optimizer_tail.state_dict(),
    'epoch': epoch},
    osp.join(SAVE_DIR, 'ep{:03d}.pth'.format(epoch+1))
)

In [67]:
SAVE_DIR = 'logs/Tendency/mymodel/CIFAR10/exp-0.1' 
model_head = ResNet18(num_classes)
model_tail = ResNet18(num_classes)
model_head.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model_head'])
model_tail.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model_tail'])
model_head = model_head.cuda()
model_tail = model_tail.cuda()

topk_acc, head_acc_ok, tail_acc = compute_accuracy(test_loader, model_head)
topk_acc, head_acc, tail_acc_ok = compute_accuracy(test_loader, model_tail)
topk_acc = (head_acc_ok[0]+ tail_acc_ok[0])/2
tail_acc = tail_acc_ok[0]
head_acc = head_acc_ok[0]
print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc, head_acc, tail_acc))

Acc 80.36 	 AccHead 80.56 	 AccTail 80.16
